In [3]:
class Variable():
    __counter = 0
    def __init__(self,data,is_leaf=True,backward_fun=None):
        if backward_fun is None and not is_leaf:
            raise ValueError('non leaf nodes require backward_fun')
        self.id = Variable.__counter
        Variable.__counter += 1
        
        self.is_leaf = is_leaf
        self.prev = []
        self.backward_fun = backward_fun
        self.data = data
        self.grad = 0
        
    def backward(self):
        self.backward_fun(dy=self.grad)
    def __repr__(self):
        return f'Variable(id:{self.id}, data:{self.data}, grad:{self.grad}, prev:{list(map(lambda a:a.id,self.prev))}, is_leaf:{self.is_leaf}\n'
def plus(a,b):
    if not (isinstance(a,Variable) and isinstance(b,Variable)):
        raise ValueError('a,b needs to be a Variable instance')
    def b_fun(dy=1):
        a.grad += dy
        b.grad += dy
    res = Variable(a.data+b.data,is_leaf=False,backward_fun=b_fun)
    res.prev.extend([a,b])
    return res

def multiply(a,b):
    if not (isinstance(a,Variable) and isinstance(b,Variable)):
            raise ValueError('a,b needs to be a Variable instance')
    def b_fun(dy=1):
        a.grad += b.data*dy
        b.grad += a.data*dy
    res = Variable(a.data+b.data,is_leaf=False,backward_fun=b_fun)
    res.prev.extend([a,b])
    return res

def c_multiply(a,c):
    if not (isinstance(a,Variable) and isinstance(c,(int, float))):
        raise ValueError('a needs to be a Variable, c needs to be one of (int, float)')
    def b_fun(dy=1):
        a.grad += c*dy
    res = Variable(a.data*c,is_leaf=False,backward_fun=b_fun)
    res.prev.append(a)
    return res

def top_sort(var):
    vars_seen = set()
    top_sort = []
    def top_sort_helper(vr):
        if (vr in vars_seen) or vr.is_leaf:
            pass
        else:
            vars_seen.add(vr)
            for pvar in vr.prev:
                top_sort_helper(pvar)
            top_sort.append(vr)    
    top_sort_helper(var)
    return top_sort

def backward_graph(var):
    if not isinstance(var,Variable):
        raise ValueError('var needs to be a Variable instance')
    tsorted = top_sort(var)
    var.grad=1
    for var in reversed(tsorted):
        var.backward()

In [4]:
l1 = Variable(2)
l2 = Variable(3)
print(l1,l2)

n1 = c_multiply(l1,2)
n2 = plus(n1,l2)
n3 = multiply(n2,n2)
# print(c,d)
# t_sort = top_sort(d)
# print(t_sort)
backward_graph(n3)
print(l1,l2,n1,n2,n3)

Variable(id:0, data:2, grad:0, prev:[], is_leaf:True
 Variable(id:1, data:3, grad:0, prev:[], is_leaf:True

Variable(id:0, data:2, grad:28, prev:[], is_leaf:True
 Variable(id:1, data:3, grad:14, prev:[], is_leaf:True
 Variable(id:2, data:4, grad:14, prev:[0], is_leaf:False
 Variable(id:3, data:7, grad:14, prev:[2, 1], is_leaf:False
 Variable(id:4, data:14, grad:1, prev:[3, 3], is_leaf:False

